In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
DATA_DIR = '../../datasets/NNN/'

all_ks = {}
to_remove = []
for _r in ['face', 'body', 'object']:
    subdir = os.path.join(DATA_DIR, f'{_r}_mins.pkl')
    df = pd.read_pickle(subdir)
    all_ks.update(df)

    SAVE_DIR = '../../datasets/NNN/'
    ed_df = pd.read_pickle(os.path.join(DATA_DIR, f'{_r}_ed.pkl'))
    # find ROIs that have any NaN ED
    bad_rois = (
        ed_df
        .groupby('ROI')['ED']
        .apply(lambda x: x.isna().any())
    )
    
    to_remove.extend(list(bad_rois[bad_rois].index))
    
dat = (
    pd.DataFrame.from_dict(all_ks, orient='index', columns=['k', 'score'])
    .reset_index()
    .rename(columns={'index': 'ROI'})
)

# keep only ROIs with no NaN EDs
dat = dat[~dat['ROI'].isin(to_remove)].reset_index(drop=True)

dat['Group'] = dat['ROI'].str[-1]   # F, B, O
dat['Group'].value_counts()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6, 4))

sns.boxplot(data=dat, x='Group', y='k',
            fill=False, color='black', ax=ax)
sns.stripplot(data=dat, x='Group', y='k',
            color='red', alpha=0.8, ax=ax)

ax.set_ylabel('k value')
ax.set_xlabel('')
ax.set_title('optimal k per ROI')
# ax.tick_params(axis='x', labelrotation=45)

sns.despine(fig=fig, offset=5, trim=True)
plt.show()